### FAISS

Facebook AI Similarity Search (FAISS) is a library for efficint similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, upto ones that possibly do not fit in RAM. it also contain supporting code for evaluation and parameter tuning.

In [2]:
from langchain_community.document_loaders import TextLoader 
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter 

loader = TextLoader('quantum_wiki.txt')
documents=loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs=text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'quantum_wiki.txt'}, page_content='A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior using specialized hardware. Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster[a] than any modern "classical" computer. Theoretically a large-scale quantum computer could break some widely used encryption schemes and aid physicists in performing physical simulations; however, the current state of the art is largely experimental and impractical, with several obstacles to useful applications.'),
 Document(metadata={'source': 'quantum_wiki.txt'}, page_content='The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a c

In [4]:
embeddings=OllamaEmbeddings(model="Gemma:2b")
db=FAISS.from_documents(docs,embeddings)
db

C:\Users\Admin\AppData\Local\Temp\ipykernel_17868\2597715993.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="Gemma:2b")


In [5]:
query = "What is the basic unit of information in quantum computing"
docs = db.similarity_search(query)
docs[0].page_content

'The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, which loosely means that it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.'

### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely works with retrievers.

In [6]:
retriever=db.as_retriever()
retriever.invoke(query)
docs[0].page_content

'The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, which loosely means that it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.'

### Similarity Search With Score

There are some FAISS specific methods. One of them is similarity_search_wwith_score, which allows you to return not only the documents but also the score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [7]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'quantum_wiki.txt'}, page_content='The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, which loosely means that it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.'),
  3546.8018),
 (Document(metadata={'source': 'quantum_wiki.txt'}, page_content='A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particle

In [8]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-1.3245887756347656,
 -3.6367485523223877,
 -1.8172954320907593,
 1.8474563360214233,
 0.9959392547607422,
 0.9379947185516357,
 -0.9593826532363892,
 -0.7032149434089661,
 -1.1398746967315674,
 -1.4576654434204102,
 0.8461337685585022,
 0.04152447357773781,
 0.4498867392539978,
 0.24791061878204346,
 0.5016061067581177,
 -0.0683259516954422,
 2.2341268062591553,
 0.5842310786247253,
 1.498003602027893,
 0.19790367782115936,
 1.6621447801589966,
 0.4935525357723236,
 -0.18055136501789093,
 0.7732419371604919,
 -0.0714515894651413,
 0.32676154375076294,
 1.3541841506958008,
 0.01960907131433487,
 1.445173740386963,
 -1.2139798402786255,
 0.13920164108276367,
 -0.6266161203384399,
 -0.050102781504392624,
 -0.9091617465019226,
 -1.547741413116455,
 -1.4480185508728027,
 0.3315621316432953,
 0.7776921391487122,
 1.786895513534546,
 0.2407728135585785,
 0.779673159122467,
 0.46632686257362366,
 0.4030246436595917,
 1.15896475315094,
 -0.5800496935844421,
 1.3298510313034058,
 0.86187821626

In [9]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_and_score

[(Document(metadata={'source': 'quantum_wiki.txt'}, page_content='The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, which loosely means that it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.'),
  3546.8018),
 (Document(metadata={'source': 'quantum_wiki.txt'}, page_content='A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particle

## Saving and Loading

In [10]:
db.save_local("Faiss_index")

In [11]:
new_db = FAISS.load_local("Faiss_index",embeddings,allow_dangerous_deserialization=True)
docss=new_db.similarity_search(query)
docss

[Document(metadata={'source': 'quantum_wiki.txt'}, page_content='The basic unit of information in quantum computing, the qubit (or "quantum bit"), serves the same function as the bit in classical computing. However, unlike a classical bit, which can be in one of two states (a binary), a qubit can exist in a superposition of its two "basis" states, which loosely means that it is in both states simultaneously. When measuring a qubit, the result is a probabilistic output of a classical bit. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.'),
 Document(metadata={'source': 'quantum_wiki.txt'}, page_content='A quantum computer is a computer that exploits quantum mechanical phenomena. On small scales, physical matter exhibits properties of both particles and waves, and